In [1]:
from diffusers import StableDiffusionPipeline
import gradio as gr
import torch

In [2]:

# Load model
#model_id = "stabilityai/stable-diffusion-2-1"
model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [3]:

# Function to generate mockup image from text prompt
def generate_mockup(prompt, num_inference_steps=50, guidance_scale=7.5):
    if device == "cuda":
        image = pipe(
            prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale
        ).images[0]
    else:
        with torch.no_grad():
            image = pipe(
                prompt,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale
            ).images[0]

    # Simple info text under the image
    info = f"Image Size: {image.size[0]}×{image.size[1]} — Model: Stable Diffusion-v1-5"

    return image, info



In [4]:
# Custom Theme
theme = gr.themes.Soft(
    primary_hue="pink",
    secondary_hue="gray",
    neutral_hue="slate",
    font=["Inter", "sans-serif"],
).set(
    button_primary_background_fill="linear-gradient(90deg, #6366F1, #3B82F6)",
    button_primary_text_color="white",
    button_primary_background_fill_hover="linear-gradient(90deg, #4F46E5, #2563EB)",
    button_border_width="0px",
)

# Interface
with gr.Blocks(theme=theme, css="""
#title {text-align:center; font-size:1.8em; font-weight:700; margin-bottom:0.4em;}
#subtitle {text-align:center; color:#6B7280; font-size:1em; margin-bottom:1.2em;}
.footer {text-align:center; font-size:0.85em; color:#9CA3AF; margin-top:1.5em;}
""") as demo:

    gr.Markdown("<div id='title'>AI Product Mockup Generator</div>")
    gr.Markdown("<div id='subtitle'>Turn your text ideas into professional product mockups instantly!</div>")

    with gr.Row():
        with gr.Column(scale=1):
            prompt = gr.Textbox(
                label="Enter your product description",
                placeholder="e.g., A modern smartwatch on a white background",
                lines=2
            )

            # Advanced settings hidden under an accordion
            with gr.Accordion("Advanced Settings", open=False):
                steps = gr.Slider(1, 50, value=10, step=5, label="Inference Steps")
                guidance = gr.Slider(1.0, 15.0, value=7.5, step=0.1, label="Guidance Scale")

            with gr.Row():
                generate_btn = gr.Button("Generate Mockup", variant="primary")
                clear_btn = gr.Button("Clear")

        with gr.Column(scale=1):
            output_image = gr.Image(type="pil", label="Generated Mockup", show_label=False)
            image_info = gr.Markdown("")

    # Button functions
    generate_btn.click(generate_mockup, [prompt, steps, guidance], [output_image, image_info])

    # Clear button resets everything
    clear_btn.click(lambda: ("", None, ""), None, [prompt, output_image, image_info])

    #gr.Markdown("<div class='footer'>Created with Stable Diffusion 2.1 and Gradio</div>")

# Launch App
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bce5d535c13c4e4f2f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
